WPP2024_GEN_F01_DEMOGRAPHIC_INDICATORS_COMPACT.xlsx  
hdro_indicators_aggregates_sdn.csv  
hdro_indicators_sdn.csv  
indicators_sdn.csv  
sdg_indicatorlist_sdn.csv  
sdg_data_sdn.csv  
gender_sdn.csv  

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
from google.colab import drive
drive.mount('/content/MyDrive')

Mounted at /content/MyDrive


In [3]:
!ls ./MyDrive/MyDrive/omneda_sudan/

dtm_sdn_smu-bi-weekly-13-_-17122024_v02_public_hdx.xlsx
education_sdn.csv
gender_sdn.csv
hdro_indicators_aggregates_sdn.csv
hdro_indicators_sdn.csv
indicators_sdn.csv
primary-secondary-enrollment-completion-rates.csv
sdg_data_sdn.csv
sdg_indicatorlist_sdn.csv
WPP2024_GEN_F01_DEMOGRAPHIC_INDICATORS_COMPACT.xlsx
WPP2024_GEN_F01_DEMOGRAPHIC_INDICATORS.xlsx


### Read in the United Nations demographic indicator data

###### WPP2024_GEN_F01_DEMOGRAPHIC_INDICATORS_COMPACT.xlsx

In [4]:
demo_indicator = pd.read_excel('./MyDrive/MyDrive/omneda_sudan/WPP2024_GEN_F01_DEMOGRAPHIC_INDICATORS_COMPACT.xlsx',
                               sheet_name="Estimates",
                               skiprows=15, header=[0,1])

In [5]:
demo_indicator.head()

Unnamed: 0_level_0 Unnamed: 1_level_0                   Unnamed: 2_level_0  \
               Index            Variant Region, subregion, country or area *   
0                  1          Estimates                                World   
1                  2          Estimates                                World   
2                  3          Estimates                                World   
3                  4          Estimates                                World   
4                  5          Estimates                                World   

  Unnamed: 3_level_0 Unnamed: 4_level_0 Unnamed: 5_level_0 Unnamed: 6_level_0  \
               Notes      Location code    ISO3 Alpha-code    ISO2 Alpha-code   
0                NaN                900                NaN                NaN   
1                NaN                900                NaN                NaN   
2                NaN                900                NaN                NaN   
3                NaN                900                NaN                NaN   
4                NaN                900                NaN                NaN   

  Unnamed: 7_level_0 Unnamed: 8_level_0 Unnamed: 9_level_0  ...  \
         SDMX code**               Type        Parent code  ...   
0                1.0              World                  0  ...   
1                1.0              World                  0  ...   
2                1.0              World                  0  ...   
3                1.0              World                  0  ...   
4                1.0              World                  0  ...   

                                                                      Mortality  \
  Male Mortality before Age 60 (deaths under age 60 per 1,000 male live births)   
0                                              580.5                              
1                                            566.566                              
2                                            546.444                              
3                                            535.811                              
4                                            522.058                              

                                                                                     \
  Female Mortality before Age 60 (deaths under age 60 per 1,000 female live births)   
0                                            497.388                                  
1                                            488.435                                  
2                                             475.37                                  
3                                            467.361                                  
4                                            455.621                                  

                                                                                               \
  Mortality between Age 15 and 50, both sexes (deaths under age 50 per 1,000 alive at age 15)   
0                                            238.516                                            
1                                            229.703                                            
2                                            217.311                                            
3                                            211.257                                            
4                                            203.337                                            

                                                                                              \
  Male Mortality between Age 15 and 50 (deaths under age 50 per 1,000 males alive at age 15)   
0                                            268.734                                           
1                                            256.236                                           
2                                             238.56                                           
3                                            230.96

In [6]:
# demo_indicator.columns.values
demo_indicator.columns=['_'.join(col).strip() for col in demo_indicator.columns.values]
demo_indicator.columns=[re.findall("level_0_(.*)", col)[0]  if col.startswith('Unnamed') else col for col in demo_indicator.columns ]

# demo_indicator.columns

In [7]:
demographics=demo_indicator[demo_indicator['Region, subregion, country or area *'].eq('Sudan')]


In [12]:
def mergeIntoDemographics(df, colName='Indicator Name', indexName='Year', valueName='Value', suffix=None ):

  return demographics.merge(df.pivot(columns=colName, values = valueName, index=indexName).reset_index(),
                                 left_on  = 'Year',
                                 right_on = indexName,
                                 how      = 'left',
                                 suffixes=(None, suffix)
                                 )

def cleanFile(df, yearCol='Year', indicatorCol='Indicator Name', valCol="value"):
  df.drop(0, inplace=True)
  df[yearCol]      = df[yearCol].apply(pd.to_numeric)
  df[valCol]       = df[valCol].apply(pd.to_numeric)
  df[indicatorCol] = df[indicatorCol].astype(str)
  df.drop_duplicates(subset=[yearCol, indicatorCol], inplace=True)
  return df

def printStats(filename):
  print(f"filename {filename}")
  print(f"demographics column count {len(demographics.columns)}")
  print(f"demographics shape {(demographics.shape)}")

In [9]:
print(demographics.shape)

(74, 65)


In [10]:
df=pd.read_csv('./MyDrive/MyDrive/omneda_sudan/hdro_indicators_aggregates_sdn.csv')
yearCol = 'year'
indicatorCol = 'indicator_name'
valCol = 'value'
df=cleanFile(df, yearCol, indicatorCol, valCol)
demographics = mergeIntoDemographics(df, indicatorCol, yearCol, valCol, 'hdro')
print(f"demographics column count {len(demographics.columns)}")
print(f"demographics shape {(demographics.shape)}")

demographics column count 71
demographics shape (74, 71)


In [11]:
filename= "hdro_indicators_sdn.csv"
df=pd.read_csv(f"./MyDrive/MyDrive/omneda_sudan/{filename}")
yearCol = 'year'
indicatorCol = 'indicator_name'
valCol = 'value'
df=cleanFile(df, yearCol, indicatorCol, valCol)
demographics = mergeIntoDemographics(df, indicatorCol, yearCol, valCol, 'hdro')

print(f"merged {filename}")
print(f"demographics column count {len(demographics.columns)}")
print(f"demographics shape {(demographics.shape)}")


mergd hdro_indicators_sdn.csv
demographics column count 117
demographics shape (74, 117)


In [13]:
filename= "indicators_sdn.csv"
df=pd.read_csv(f"./MyDrive/MyDrive/omneda_sudan/{filename}")
yearCol = 'Year'
indicatorCol = 'Indicator Name'
valCol = 'Value'
df=cleanFile(df, yearCol, indicatorCol, valCol)
demographics = mergeIntoDemographics(df, indicatorCol, yearCol, valCol, 'indicators')
printStats(filename)

filename indicators_sdn.csv
demographics column count 3000
demographics shape (74, 3000)


In [15]:
filename = "sdg data"
sdg      = pd.read_csv('./MyDrive/MyDrive/omneda_sudan/sdg_indicatorlist_sdn.csv')
sdg_data = pd.read_csv('./MyDrive/MyDrive/omneda_sudan/sdg_data_sdn.csv')
df       = sdg_data.merge(sdg, on='indicator_id', how='left')


yearCol = 'year'
indicatorCol = 'indicator_label_en'
valCol = 'value'

df=cleanFile(df, yearCol, indicatorCol, valCol)
demographics = mergeIntoDemographics(df, indicatorCol, yearCol, valCol, 'sdg')
# print(f"demographics column count {len(demographics.columns)}")
# print(demographics.shape)
printStats(filename)


filename sdg data
demographics column count 4051
demographics shape (74, 4051)


In [16]:
filename= "gender_sdn.csv"
df=pd.read_csv(f"./MyDrive/MyDrive/omneda_sudan/{filename}")

yearCol = 'Year'
indicatorCol = 'Indicator Name'
valCol = 'Value'
df=cleanFile(df, yearCol, indicatorCol, valCol)
demographics = mergeIntoDemographics(df, indicatorCol, yearCol, valCol, 'gender')
# print(f"demographics column count {len(demographics.columns)}")
# print(demographics.shape)
printStats(filename)


filename gender_sdn.csv
demographics column count 4271
demographics shape (74, 4271)


In [17]:
filename= "education_sdn.csv"
df=pd.read_csv(f'./MyDrive/MyDrive/omneda_sudan/{filename}')
yearCol = 'Year'
indicatorCol = 'Indicator Name'
valCol = 'Value'
df=cleanFile(df, yearCol, indicatorCol, valCol)
demographics = mergeIntoDemographics(df, indicatorCol, yearCol, valCol, 'education')
# print(f"demographics column count {len(demographics.columns)}")
# print(demographics.shape)
printStats(filename)


filename education_sdn.csv
demographics column count 5055
demographics shape (74, 5055)


In [18]:
df['Value'] = df['Value'].apply(pd.to_numeric)

In [19]:
demographics.to_csv('demographics.csv')

In [20]:
demo_cols = pd.DataFrame(demographics.columns)

In [21]:
demo_cols.to_csv('demo_cols.csv')

In [22]:
demographics.head()

,Index,Variant,"Region, subregion, country or area *",Notes,Location code,ISO3 Alpha-code,ISO2 Alpha-code,SDMX code**,Type,Parent code,...,"Trained teachers in upper secondary education, female (% of female teachers)education","Trained teachers in upper secondary education, male (% of male teachers)education","Unemployment, female (% of female labor force) (modeled ILO estimate)education","Unemployment, male (% of male labor force) (modeled ILO estimate)education","Unemployment, total (% of total labor force) (modeled ILO estimate)education","Vocational and Technical enrolment (% of total secondary enrolment), femaleeducation","Youth illiterate population, 15-24 years, % femaleeducation","Youth illiterate population, 15-24 years, both sexes (number)education","Youth illiterate population, 15-24 years, female (number)education","Youth illiterate population, 15-24 years, male (number)education"
0,5185,Estimates,Sudan,NaN,729,SDN,SD,729.0,Country/Area,912,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5186,Estimates,Sudan,NaN,729,SDN,SD,729.0,Country/Area,912,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5187,Estimates,Sudan,NaN,729,SDN,SD,729.0,Country/Area,912,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5188,Estimates,Sudan,NaN,729,SDN,SD,729.0,Country/Area,912,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5189,Estimates,Sudan,NaN,729,SDN,SD,729.0,Country/Area,912,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
demographics.dtypes

,0
Index,int64
Variant,object
"Region, subregion, country or area *",object
Notes,object
Location code,int64
...,...
"Vocational and Technical enrolment (% of total secondary enrolment), femaleeducation",float64
"Youth illiterate population, 15-24 years, % femaleeducation",float64
"Youth illiterate population, 15-24 years, both sexes (number)education",float64
"Youth illiterate population, 15-24 years, female (number)education",float64


In [24]:
id_vars = demographics.columns[:11].to_list()
id_vars

['Index',
 'Variant',
 'Region, subregion, country or area *',
 'Notes',
 'Location code',
 'ISO3 Alpha-code',
 'ISO2 Alpha-code',
 'SDMX code**',
 'Type',
 'Parent code',
 'Year']

In [25]:
demographics2 = demographics.melt(id_vars=id_vars, var_name='indicator', value_name='value')
# demographics2['Year'] = demographics2['Year'].astype(str)
demographics2.head()

,Index,Variant,"Region, subregion, country or area *",Notes,Location code,ISO3 Alpha-code,ISO2 Alpha-code,SDMX code**,Type,Parent code,Year,indicator,value
0,5185,Estimates,Sudan,NaN,729,SDN,SD,729.0,Country/Area,912,1950.0,"Population_Total Population, as of 1 January (...",6098.654
1,5186,Estimates,Sudan,NaN,729,SDN,SD,729.0,Country/Area,912,1951.0,"Population_Total Population, as of 1 January (...",6281.167
2,5187,Estimates,Sudan,NaN,729,SDN,SD,729.0,Country/Area,912,1952.0,"Population_Total Population, as of 1 January (...",6470.953
3,5188,Estimates,Sudan,NaN,729,SDN,SD,729.0,Country/Area,912,1953.0,"Population_Total Population, as of 1 January (...",6666.266
4,5189,Estimates,Sudan,NaN,729,SDN,SD,729.0,Country/Area,912,1954.0,"Population_Total Population, as of 1 January (...",6865.544


In [26]:
demographics2['value']=demographics2['value'].apply(pd.to_numeric,  errors='coerce')
demographics2.dtypes

,0
Index,int64
Variant,object
"Region, subregion, country or area *",object
Notes,object
Location code,int64
ISO3 Alpha-code,object
ISO2 Alpha-code,object
SDMX code**,float64
Type,object
Parent code,int64


In [27]:
demographics2.shape

(373256, 13)

In [28]:
demographics2.columns

Index(['Index', 'Variant', 'Region, subregion, country or area *', 'Notes',
       'Location code', 'ISO3 Alpha-code', 'ISO2 Alpha-code', 'SDMX code**',
       'Type', 'Parent code', 'Year', 'indicator', 'value'],
      dtype='object')

In [29]:
demographics2=demographics2.reset_index().set_index(['index']+id_vars+['indicator']).unstack('Year').reset_index()
demographics2.head()

index Index    Variant Region, subregion, country or area * Notes  \
Year                                                                     
0        0  5185  Estimates                                Sudan   NaN   
1        1  5186  Estimates                                Sudan   NaN   
2        2  5187  Estimates                                Sudan   NaN   
3        3  5188  Estimates                                Sudan   NaN   
4        4  5189  Estimates                                Sudan   NaN   

     Location code ISO3 Alpha-code ISO2 Alpha-code SDMX code**          Type  \
Year                                                                           
0              729             SDN              SD       729.0  Country/Area   
1              729             SDN              SD       729.0  Country/Area   
2              729             SDN              SD       729.0  Country/Area   
3              729             SDN              SD       729.0  Country/Area   
4              729             SDN              SD       729.0  Country/Area   

      ...  value                                                          \
Year  ... 2014.0 2015.0 2016.0 2017.0 2018.0 2019.0 2020.0 2021.0 2022.0   
0     ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
1     ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2     ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
3     ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
4     ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   

             
Year 2023.0  
0       NaN  
1       NaN  
2       NaN  
3       NaN  
4       NaN  

[5 rows x 86 columns]

In [30]:
len(demographics2.columns)

86

In [31]:
demographics2.columns = [col[1] if col[0]=='value' else col[0]  for col in demographics2.columns.values]
demographics2.head()

,index,Index,Variant,"Region, subregion, country or area *",Notes,Location code,ISO3 Alpha-code,ISO2 Alpha-code,SDMX code**,Type,...,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023.0
0,0,5185,Estimates,Sudan,NaN,729,SDN,SD,729.0,Country/Area,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,5186,Estimates,Sudan,NaN,729,SDN,SD,729.0,Country/Area,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,5187,Estimates,Sudan,NaN,729,SDN,SD,729.0,Country/Area,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,5188,Estimates,Sudan,NaN,729,SDN,SD,729.0,Country/Area,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5189,Estimates,Sudan,NaN,729,SDN,SD,729.0,Country/Area,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
demographics2.columns[12:]

Index([1950.0, 1951.0, 1952.0, 1953.0, 1954.0, 1955.0, 1956.0, 1957.0, 1958.0,
       1959.0, 1960.0, 1961.0, 1962.0, 1963.0, 1964.0, 1965.0, 1966.0, 1967.0,
       1968.0, 1969.0, 1970.0, 1971.0, 1972.0, 1973.0, 1974.0, 1975.0, 1976.0,
       1977.0, 1978.0, 1979.0, 1980.0, 1981.0, 1982.0, 1983.0, 1984.0, 1985.0,
       1986.0, 1987.0, 1988.0, 1989.0, 1990.0, 1991.0, 1992.0, 1993.0, 1994.0,
       1995.0, 1996.0, 1997.0, 1998.0, 1999.0, 2000.0, 2001.0, 2002.0, 2003.0,
       2004.0, 2005.0, 2006.0, 2007.0, 2008.0, 2009.0, 2010.0, 2011.0, 2012.0,
       2013.0, 2014.0, 2015.0, 2016.0, 2017.0, 2018.0, 2019.0, 2020.0, 2021.0,
       2022.0, 2023.0],
      dtype='object')

In [33]:
['index'] + id_vars[:-1]+ ['indicator']

['index',
 'Index',
 'Variant',
 'Region, subregion, country or area *',
 'Notes',
 'Location code',
 'ISO3 Alpha-code',
 'ISO2 Alpha-code',
 'SDMX code**',
 'Type',
 'Parent code',
 'indicator']

In [34]:
id_vars[2:-1]

['Region, subregion, country or area *',
 'Notes',
 'Location code',
 'ISO3 Alpha-code',
 'ISO2 Alpha-code',
 'SDMX code**',
 'Type',
 'Parent code']

In [35]:
'Region, subregion, country or area *',  'Notes',  'Location code',  'ISO3 Alpha-code',  'ISO2 Alpha-code',  'SDMX code**','Type', 'Parent code'

('Region, subregion, country or area *',
 'Notes',
 'Location code',
 'ISO3 Alpha-code',
 'ISO2 Alpha-code',
 'SDMX code**',
 'Type',
 'Parent code')

In [36]:
demographics2.groupby(  id_vars[2:-1]+ ['indicator'])[demographics2.columns[12:]].sum()

,,,,,,,,,1950.0,1951.0,1952.0,1953.0,1954.0,1955.0,1956.0,1957.0,1958.0,1959.0,...,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023.0
"Region, subregion, country or area *",Notes,Location code,ISO3 Alpha-code,ISO2 Alpha-code,SDMX code**,Type,Parent code,indicator,,,,,,,,,,,,,,,,,,,,,


In [37]:
# demographics2.groupby(   ['Parent code','Location code','indicator'])[demographics2.columns[12:]].sum()
demographicsAlt = demographics2.groupby(   [  'Region, subregion, country or area *' , 'Location code',  'ISO3 Alpha-code',  'ISO2 Alpha-code',  'Type', 'Parent code','indicator'])[demographics2.columns[12:]].sum().reset_index()
demographicsAlt.head()

,"Region, subregion, country or area *",Location code,ISO3 Alpha-code,ISO2 Alpha-code,Type,Parent code,indicator,1950.0,1951.0,1952.0,...,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023.0
0,Sudan,729,SDN,SD,Country/Area,912,Administration of a nationally representative...,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
1,Sudan,729,SDN,SD,Country/Area,912,Administration of a nationally representative...,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
2,Sudan,729,SDN,SD,Country/Area,912,Administration of a nationally-representative...,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Sudan,729,SDN,SD,Country/Area,912,Administration of a nationally-representative...,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Sudan,729,SDN,SD,Country/Area,912,Administration of a nationally-representative...,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
# demographicsAlt.to_csv('demographicsAlt.csv')

In [52]:
 ~demographicsAlt['indicator'].str.contains('female')

,indicator
0,True
1,True
2,True
3,True
4,True
...,...
5039,True
5040,True
5041,True
5042,True


In [136]:
condlist = [
    demographicsAlt['indicator'].str.contains('female'),
    demographicsAlt['indicator'].str.contains('male') & (~demographicsAlt['indicator'].str.contains('female'))
    ]
choicelist = ['female', 'male']
demographicsAlt['gender'] = np.select(condlist, choicelist, default='unknown')

demographicsAlt['quintile']   = demographicsAlt['indicator'].str.extract(r"(\w+)\s+quintile", re.IGNORECASE)
demographicsAlt['ages']       = demographicsAlt['indicator'].str.extract(r"\bage[s]?\s([\d|\-|\+]*[\band above|\sand\d|months]*)", re.IGNORECASE)
demographicsAlt['percentage'] = demographicsAlt['indicator'].str.extract(r"\((\%)" , flags=re.IGNORECASE)

# condlist2 = [
#     demographicsAlt['indicator'].str.match("(\w+)\s+County", case=False),
# ]
# choicelist2 = ['County']
# demographicsAlt['county'] = np.select(condlist2, choicelist2, default='unknown')

# demographicsAlt['quintile'] = demographicsAlt['indicator'].str.match("(\w+)\s+quintile", case=False)

In [141]:
# demographicsAlt[demographicsAlt['indicator'].str.extract(r"\((\%)" , flags=re.IGNORECASE )]

In [140]:
# demographicsAlt[~demographicsAlt['percentage'].isnull()]

In [143]:
demographicsAlt.insert(7, "gender", demographicsAlt.pop("gender"))
demographicsAlt.insert(8, "quintile", demographicsAlt.pop("quintile"))
demographicsAlt.insert(9, "ages", demographicsAlt.pop("ages"))
demographicsAlt.insert(10, "percentage", demographicsAlt.pop("percentage"))

In [145]:
demographicsAlt.to_csv('demographicsAlt.csv')

In [61]:
# demographics2.reset_index().set_index(['index','indicator']+id_vars ).unstack('Year')

In [30]:
# df=pd.read_csv('./MyDrive/MyDrive/omneda_sudan/indicators_sdn.csv')
# df

,Country Name,Country ISO3,Year,Indicator Name,Indicator Code,Value
0,#country+name,#country+code,#date+year,#indicator+name,#indicator+code,#indicator+value+num
1,Sudan,SDN,2022,Fertilizer consumption (kilograms per hectare ...,AG.CON.FERT.ZS,7.10201554286672
2,Sudan,SDN,2021,Fertilizer consumption (kilograms per hectare ...,AG.CON.FERT.ZS,7.10307199292779
3,Sudan,SDN,2020,Fertilizer consumption (kilograms per hectare ...,AG.CON.FERT.ZS,13.847366781552
4,Sudan,SDN,2019,Fertilizer consumption (kilograms per hectare ...,AG.CON.FERT.ZS,6.46987402618929
...,...,...,...,...,...,...
67539,Sudan,SDN,1981,Travel services (% of commercial service exports),TX.VAL.TRVL.ZS.WT,17.4276555279644
67540,Sudan,SDN,1980,Travel services (% of commercial service exports),TX.VAL.TRVL.ZS.WT,20.926517571885
67541,Sudan,SDN,1979,Travel services (% of commercial service exports),TX.VAL.TRVL.ZS.WT,17.0220749166778
67542,Sudan,SDN,1978,Travel services (% of commercial service exports),TX.VAL.TRVL.ZS.WT,12.2330586272349


In [39]:
# df['Indicator Name'] = df['Indicator Name'].astype(str)

In [50]:
# df[df['Indicator Name'].str.contains('population,')]

,Country Name,Country ISO3,Year,Indicator Name,Indicator Code,Value
22702,Sudan,SDN,2020,"School age population, pre-primary education, ...",SP.PRE.TOTL.FE.IN,1175180
22703,Sudan,SDN,2019,"School age population, pre-primary education, ...",SP.PRE.TOTL.FE.IN,1159254
22704,Sudan,SDN,2018,"School age population, pre-primary education, ...",SP.PRE.TOTL.FE.IN,1143265
22705,Sudan,SDN,2017,"School age population, pre-primary education, ...",SP.PRE.TOTL.FE.IN,1127140
22706,Sudan,SDN,2016,"School age population, pre-primary education, ...",SP.PRE.TOTL.FE.IN,1110886
...,...,...,...,...,...,...
59164,Sudan,SDN,1963,"Age population, age 05, male",SP.POP.AG05.MA.IN,152975
59165,Sudan,SDN,1962,"Age population, age 05, male",SP.POP.AG05.MA.IN,147943
59166,Sudan,SDN,1961,"Age population, age 05, male",SP.POP.AG05.MA.IN,143150
59167,Sudan,SDN,1960,"Age population, age 05, male",SP.POP.AG05.MA.IN,138560.5


In [51]:
# demographics.value_counts()

Series([], Name: count, dtype: int64)

In [52]:
# df.T

,0,1,2,3,4,5,6,7,8,9,...,67534,67535,67536,67537,67538,67539,67540,67541,67542,67543
Country Name,#country+name,Sudan,Sudan,Sudan,Sudan,Sudan,Sudan,Sudan,Sudan,Sudan,...,Sudan,Sudan,Sudan,Sudan,Sudan,Sudan,Sudan,Sudan,Sudan,Sudan
Country ISO3,#country+code,SDN,SDN,SDN,SDN,SDN,SDN,SDN,SDN,SDN,...,SDN,SDN,SDN,SDN,SDN,SDN,SDN,SDN,SDN,SDN
Year,#date+year,2022,2021,2020,2019,2018,2017,2016,2015,2014,...,1986,1985,1984,1983,1982,1981,1980,1979,1978,1977
Indicator Name,#indicator+name,Fertilizer consumption (kilograms per hectare ...,Fertilizer consumption (kilograms per hectare ...,Fertilizer consumption (kilograms per hectare ...,Fertilizer consumption (kilograms per hectare ...,Fertilizer consumption (kilograms per hectare ...,Fertilizer consumption (kilograms per hectare ...,Fertilizer consumption (kilograms per hectare ...,Fertilizer consumption (kilograms per hectare ...,Fertilizer consumption (kilograms per hectare ...,...,Travel services (% of commercial service exports),Travel services (% of commercial service exports),Travel services (% of commercial service exports),Travel services (% of commercial service exports),Travel services (% of commercial service exports),Travel services (% of commercial service exports),Travel services (% of commercial service exports),Travel services (% of commercial service exports),Travel services (% of commercial service exports),Travel services (% of commercial service exports)
Indicator Code,#indicator+code,AG.CON.FERT.ZS,AG.CON.FERT.ZS,AG.CON.FERT.ZS,AG.CON.FERT.ZS,AG.CON.FERT.ZS,AG.CON.FERT.ZS,AG.CON.FERT.ZS,AG.CON.FERT.ZS,AG.CON.FERT.ZS,...,TX.VAL.TRVL.ZS.WT,TX.VAL.TRVL.ZS.WT,TX.VAL.TRVL.ZS.WT,TX.VAL.TRVL.ZS.WT,TX.VAL.TRVL.ZS.WT,TX.VAL.TRVL.ZS.WT,TX.VAL.TRVL.ZS.WT,TX.VAL.TRVL.ZS.WT,TX.VAL.TRVL.ZS.WT,TX.VAL.TRVL.ZS.WT
Value,#indicator+value+num,7.10201554286672,7.10307199292779,13.847366781552,6.46987402618929,7.10307199292779,8.33866539841623,9.38239636987982,4.47432480348684,4.20062593451296,...,11.9910514541387,19.7315253751551,22.3521550283997,11.2576064908722,12.1516195322727,17.4276555279644,20.926517571885,17.0220749166778,12.2330586272349,11.1112816463425


In [ ]:
# df = pd.read_csv('./MyDrive/MyDrive/omneda_sudan/primary-secondary-enrollment-completion-rates.csv')

In [ ]:
# df[df['Entity'] =='Sudan']

### Read Aggregated Human Development Indicators for Sudan
hdro_indicators_aggregates_sdn.csv

In [ ]:
# hdro_indicators_agg=pd.read_csv('./MyDrive/MyDrive/omneda_sudan/hdro_indicators_aggregates_sdn.csv')
# hdro_indicators_agg.drop(0, inplace=True)
# hdro_indicators_agg['year'] = hdro_indicators_agg['year'].apply(pd.to_numeric)

# hdro_indicators_agg.head()

In [ ]:
# hdro_indicators.pivot(columns=['indicator_name'], values = 'value', index=['year']).reset_index().head()

### merge into demographics

In [ ]:
# demographics= demographics.merge(hdro_indicators_agg.pivot(columns=['indicator_name'], values = 'value', index=['year']).reset_index(),
#                                  left_on='Year',
#                                  right_on='year',
#                                  how='left'
#                                  ).drop(columns=('year'))

In [ ]:
# demographics.columns

#### hdro_indicators_sdn.csv

In [ ]:
# hdro_indicators=pd.read_csv('./MyDrive/MyDrive/omneda_sudan/hdro_indicators_sdn.csv')
# hdro_indicators.drop(0, inplace=True)
# hdro_indicators['year'] = hdro_indicators['year'].apply(pd.to_numeric)

# hdro_indicators.head()

In [ ]:
# demographics= demographics.merge(hdro_indicators.pivot(columns=['indicator_name'], values = 'value', index=['year']).reset_index(),
#                                  left_on='Year',
#                                  right_on='year',
#                                  how='left'
#                                  ).drop(columns=('year'))

In [ ]:
# hdro_indicators.pivot(columns=['indicator_name'], values = 'value', index=['year']).reset_index().head()

In [ ]:
# hdro_indicators.columns

In [ ]:
# hdro_indicators.iloc[25:].head(10)

In [ ]:
# hdro_indicators['index_indicator_name']=hdro_indicators['index_name'] + "_" + hdro_indicators['indicator_name']

In [ ]:
# hdro_indicators.pivot(columns=  ['index_indicator_name'], values = 'value', index=['year']).reset_index().head()

In [ ]:
# demographics.head().to_csv('demographics_top5.csv')

#### Indicators SDN
hdro_indicators=pd.read_csv('./MyDrive/MyDrive/omneda_sudan/hdro_indicators_sdn.csv')
hdro_indicators.head()

In [ ]:
# indicators=pd.read_csv('./MyDrive/MyDrive/omneda_sudan/indicators_sdn.csv')
# indicators.drop(0, inplace=True)
# indicators.drop_duplicates(subset=['Year', 'Indicator Name'], inplace=True)
# # indicators.drop_duplicates(subset=['Year', 'Indicator Name', 'Value'], inplace=True)
# indicators['Year'] = indicators['Year'].apply(pd.to_numeric)
# indicators.head()

In [ ]:
# indicators.pivot(columns=['Indicator Name'], values = 'Value', index=['Year']).reset_index()

In [ ]:
# demographics= demographics.merge(indicators.pivot(columns=['Indicator Name'], values = 'Value', index=['Year']).reset_index(),
#                                  left_on='Year',
#                                  right_on='Year',
#                                  how='left'
#                                  )

#### SDG Indicator list and data

In [ ]:
# sdg=pd.read_csv('./MyDrive/MyDrive/omneda_sudan/sdg_indicatorlist_sdn.csv')
# sdg_data=pd.read_csv('./MyDrive/MyDrive/omneda_sudan/sdg_data_sdn.csv')
# sdg.iloc[10:].head()

In [ ]:

# sdg_data.head()

In [ ]:
# sdg_merge=sdg_data.merge(sdg, on='indicator_id', how='left')
# sdg_merge.drop(0, inplace=True)
# sdg_merge['year']=sdg_merge['year'].apply(pd.to_numeric)
# sdg_merge['indicator_label_en']=sdg_merge['indicator_label_en'].astype(str)
# sdg_merge.drop_duplicates(subset=['year', 'indicator_label_en'], inplace=True)
# sdg_merge.head()


In [ ]:
# sdg_merge.pivot(columns=['indicator_label_en'], values = 'value', index=['year']).reset_index().head()

In [ ]:
# demographics= demographics.merge(sdg_merge.pivot(columns=['indicator_label_en'], values = 'value', index=['year']).reset_index(),
#                                  left_on='Year',
#                                  right_on='year',
#                                  how='left'
#                                  ).drop(columns='year')

In [ ]:
# gender = pd.read_csv('./MyDrive/MyDrive/omneda_sudan/gender_sdn.csv')
# gender.drop(0, inplace=True)
# gender['Indicator Name']=gender['Indicator Name'].astype(str)
# gender['Year'] = indicators['Year'].apply(pd.to_numeric)
# gender.drop_duplicates(subset=['Year', 'Indicator Name'], inplace=True)
# gender.head()

In [ ]:
# gender.pivot(columns=['Indicator Name'], values = 'Value', index=['Year']).reset_index().head()

In [ ]:
# demographics= demographics.merge(gender.pivot(columns=['Indicator Name'], values = 'Value', index=['Year']).reset_index(),
#                                  left_on='Year',
#                                  right_on='Year',
#                                  how='left'
#                                  )
# # .drop(columns='year')

In [ ]:
# df=pd.read_csv('./MyDrive/MyDrive/omneda_sudan/education_sdn.csv')
# df=cleanFile(df, 'Year', 'Indicator Name')
# demographics = mergeIntoDemographics(df, 'Indicator Name', 'Year', 'Value', 'education')

In [ ]:
# education = pd.read_csv('./MyDrive/MyDrive/omneda_sudan/education_sdn.csv')
# education.drop(0, inplace=True)
# education['Indicator Name']=gender['Indicator Name'].astype(str)
# education['Year'] = indicators['Year'].apply(pd.to_numeric)
# gender.drop_duplicates(subset=['Year', 'Indicator Name'], inplace=True)
# education.head()

In [ ]:
# demographics= demographics.merge(education.pivot(columns=['Indicator Name'], values = 'Value', index=['Year']).reset_index(),
#                                  left_on='Year',
#                                  right_on='Year',
#                                  how='left'
#                                  )
# # .drop(columns='year')

ValueError: You are trying to merge on float64 and object columns for key 'Year'. If you wish to proceed you should use pd.concat

In [ ]:
# demographics.head()

In [ ]:
# demographics.to_csv('demographics.csv')